## 1. LIBRARIES AND SETTINGS

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.sql.functions import explode

spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

## 2. CREATING AND CLEANING DATAFRAMES

In [2]:
uniprot = spark.read.csv("DATA/uniprot links.csv", header=True, inferSchema=True)
uniprot.show()

+-----------+-------------------+-----------+----------+--------------------+
|DrugBank ID|               Name|       Type|UniProt ID|        UniProt Name|
+-----------+-------------------+-----------+----------+--------------------+
|    DB00001|          Lepirudin|BiotechDrug|    P00734|         Prothrombin|
|    DB00002|          Cetuximab|BiotechDrug|    P00533|Epidermal growth ...|
|    DB00002|          Cetuximab|BiotechDrug|    O75015|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P00736|Complement C1r su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02745|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02746|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02747|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P08637|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P09871|Complement C1s su...|
|    DB00002|          Cetuximab|BiotechDrug|    P12314|High aff

In [3]:
uniprot.count()

20941

In [4]:
species = spark.read.csv("DATA/species_13.csv", header=True, inferSchema=True, sep='\t')
species.show()

+-------------------+------------------+-----------------------+-----------------------+----------------------+----------------------+-------------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+-------------------+
|#ID(s) interactor A|ID(s) interactor B|Alt. ID(s) interactor A|Alt. ID(s) interactor B|Alias(es) interactor A|Alias(es) interactor B|Interaction detection method(s)|Publication 1st author(s)|Publication Identifier(s)|  Taxid interactor A|  Taxid interactor B| Interaction type(s)|  Source database(s)|Interaction identifier(s)|Confidence value(s)|
+-------------------+------------------+-----------------------+-----------------------+----------------------+----------------------+-------------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+

In [5]:
species.count()

1054920

Selecting useful columns 

In [6]:
species = species.select(species['#ID(s) interactor A'], species['ID(s) interactor B'], species['Confidence value(s)'])
species.show()

+-------------------+------------------+-------------------+
|#ID(s) interactor A|ID(s) interactor B|Confidence value(s)|
+-------------------+------------------+-------------------+
|   uniprotkb:P38764|  uniprotkb:P40016|intact-miscore:0.76|
|   uniprotkb:Q01939|  uniprotkb:P40016|intact-miscore:0.40|
|   uniprotkb:P33299|  uniprotkb:P40016|intact-miscore:0.69|
|   uniprotkb:Q06103|  uniprotkb:P40016|intact-miscore:0.81|
|   uniprotkb:P38764|  uniprotkb:P40016|intact-miscore:0.76|
|   uniprotkb:P40016|  uniprotkb:P38764|intact-miscore:0.76|
|   uniprotkb:P53549|  uniprotkb:P40016|intact-miscore:0.55|
|   uniprotkb:P40016|  uniprotkb:Q08723|intact-miscore:0.70|
|   uniprotkb:Q08723|  uniprotkb:P40016|intact-miscore:0.70|
|   uniprotkb:P40016|  uniprotkb:P38886|intact-miscore:0.76|
|   uniprotkb:P40016|  uniprotkb:P43588|intact-miscore:0.76|
|   uniprotkb:P40016|  uniprotkb:P38764|intact-miscore:0.76|
|   uniprotkb:P40016|  uniprotkb:Q06103|intact-miscore:0.81|
|   uniprotkb:P53008|  u

Record cleaning

In [7]:
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'uniprotkb:', ''))
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'chebi:', ''))
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'ensembl:', ''))
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'ensemblgenomes:', ''))
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'intact:', ''))
species = species.withColumn('#ID(s) interactor A', regexp_replace('#ID(s) interactor A', 'refseq:', ''))

species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'uniprotkb:', ''))
species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'chebi:', ''))
species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'ensembl:', ''))
species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'ensemblgenomes:', ''))
species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'intact:', ''))
species = species.withColumn('ID(s) interactor B', regexp_replace('ID(s) interactor B', 'refseq:', ''))

species = species.withColumn('Confidence value(s)', regexp_replace('Confidence value(s)', 'intact-miscore:', ''))

species.show()

+-------------------+------------------+-------------------+
|#ID(s) interactor A|ID(s) interactor B|Confidence value(s)|
+-------------------+------------------+-------------------+
|             P38764|            P40016|               0.76|
|             Q01939|            P40016|               0.40|
|             P33299|            P40016|               0.69|
|             Q06103|            P40016|               0.81|
|             P38764|            P40016|               0.76|
|             P40016|            P38764|               0.76|
|             P53549|            P40016|               0.55|
|             P40016|            Q08723|               0.70|
|             Q08723|            P40016|               0.70|
|             P40016|            P38886|               0.76|
|             P40016|            P43588|               0.76|
|             P40016|            P38764|               0.76|
|             P40016|            Q06103|               0.81|
|             P53008|   

Renaming columns

In [8]:
species = species.withColumnRenamed("#ID(s) interactor A", "Interactor_A")
species = species.withColumnRenamed("ID(s) interactor B", "Interactor_B")
species = species.withColumnRenamed("Confidence value(s)", "Confidence")
uniprot = uniprot.withColumnRenamed("DrugBank ID", "ID_DrugBank")
uniprot = uniprot.withColumnRenamed("UniProt ID", "ID_UniProt")

In [9]:
species.show()

+------------+------------+----------+
|Interactor_A|Interactor_B|Confidence|
+------------+------------+----------+
|      P38764|      P40016|      0.76|
|      Q01939|      P40016|      0.40|
|      P33299|      P40016|      0.69|
|      Q06103|      P40016|      0.81|
|      P38764|      P40016|      0.76|
|      P40016|      P38764|      0.76|
|      P53549|      P40016|      0.55|
|      P40016|      Q08723|      0.70|
|      Q08723|      P40016|      0.70|
|      P40016|      P38886|      0.76|
|      P40016|      P43588|      0.76|
|      P40016|      P38764|      0.76|
|      P40016|      Q06103|      0.81|
|      P53008|      P40016|      0.40|
|      P32565|      P40016|      0.40|
|      P40016|      P32565|      0.40|
|      P38764|      P40016|      0.76|
|      P53549|      P40016|      0.55|
|      P40016|      P38764|      0.76|
|      Q08723|      P40016|      0.70|
+------------+------------+----------+
only showing top 20 rows



In [10]:
uniprot.show()

+-----------+-------------------+-----------+----------+--------------------+
|ID_DrugBank|               Name|       Type|ID_UniProt|        UniProt Name|
+-----------+-------------------+-----------+----------+--------------------+
|    DB00001|          Lepirudin|BiotechDrug|    P00734|         Prothrombin|
|    DB00002|          Cetuximab|BiotechDrug|    P00533|Epidermal growth ...|
|    DB00002|          Cetuximab|BiotechDrug|    O75015|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P00736|Complement C1r su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02745|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02746|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02747|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P08637|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P09871|Complement C1s su...|
|    DB00002|          Cetuximab|BiotechDrug|    P12314|High aff

Casting string values ('Confidence' in species DF) to float

In [11]:
species = species.withColumn("Confidence", species["Confidence"].cast(FloatType()))
species.printSchema()

root
 |-- Interactor_A: string (nullable = true)
 |-- Interactor_B: string (nullable = true)
 |-- Confidence: float (nullable = true)



Dropping record in species DataFrame that have null values in Confidence column

In [12]:
species = species.na.drop(subset=["Confidence"])
species.filter(species["Confidence"].isNull()).show()

+------------+------------+----------+
|Interactor_A|Interactor_B|Confidence|
+------------+------------+----------+
+------------+------------+----------+



In [13]:
species.show()

+------------+------------+----------+
|Interactor_A|Interactor_B|Confidence|
+------------+------------+----------+
|      P38764|      P40016|      0.76|
|      Q01939|      P40016|       0.4|
|      P33299|      P40016|      0.69|
|      Q06103|      P40016|      0.81|
|      P38764|      P40016|      0.76|
|      P40016|      P38764|      0.76|
|      P53549|      P40016|      0.55|
|      P40016|      Q08723|       0.7|
|      Q08723|      P40016|       0.7|
|      P40016|      P38886|      0.76|
|      P40016|      P43588|      0.76|
|      P40016|      P38764|      0.76|
|      P40016|      Q06103|      0.81|
|      P53008|      P40016|       0.4|
|      P32565|      P40016|       0.4|
|      P40016|      P32565|       0.4|
|      P38764|      P40016|      0.76|
|      P53549|      P40016|      0.55|
|      P40016|      P38764|      0.76|
|      Q08723|      P40016|       0.7|
+------------+------------+----------+
only showing top 20 rows



In [14]:
uniprot.show()

+-----------+-------------------+-----------+----------+--------------------+
|ID_DrugBank|               Name|       Type|ID_UniProt|        UniProt Name|
+-----------+-------------------+-----------+----------+--------------------+
|    DB00001|          Lepirudin|BiotechDrug|    P00734|         Prothrombin|
|    DB00002|          Cetuximab|BiotechDrug|    P00533|Epidermal growth ...|
|    DB00002|          Cetuximab|BiotechDrug|    O75015|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P00736|Complement C1r su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02745|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02746|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P02747|Complement C1q su...|
|    DB00002|          Cetuximab|BiotechDrug|    P08637|Low affinity immu...|
|    DB00002|          Cetuximab|BiotechDrug|    P09871|Complement C1s su...|
|    DB00002|          Cetuximab|BiotechDrug|    P12314|High aff

## 3. CREATING DATAFRAME TO BE USED FOR THE RECOMMENDATION SYSTEM

The following DataFrame contains, for each drug of the uniprot DataFrame that acts with a particular Id_Uniprot protein, as many records as there are proteins (Interactor_B in the DataFrame species), having confidence values > 0.5, which interact with that specific protein ID_Uniprot (Interactor_A in the DataFrame species). Each record contains the 'Interaction' value set to 1 used as implicit feedback

In [15]:
joinedDF = uniprot.join(species, (uniprot.ID_UniProt == species.Interactor_A) & (species.Confidence >= 0.5))

joinedDF = joinedDF.select(joinedDF['ID_DrugBank'], joinedDF['ID_UniProt'], joinedDF['Interactor_B'])
joinedDF = joinedDF.withColumn("Interaction", lit(1))
joinedDF = joinedDF.filter(joinedDF.Interactor_B != "-")
joinedDF = joinedDF.dropDuplicates()

joinedDF.orderBy('ID_DrugBank', 'Interactor_B').show()

+-----------+----------+------------+-----------+
|ID_DrugBank|ID_UniProt|Interactor_B|Interaction|
+-----------+----------+------------+-----------+
|    DB00001|    P00734|  EBI-941456|          1|
|    DB00001|    P00734|      Q846V4|          1|
|    DB00002|    P00533|      A4FU49|          1|
|    DB00002|    P00533| EBI-4399559|          1|
|    DB00002|    P00533|   NP_059022|          1|
|    DB00002|    P00533|      O00170|          1|
|    DB00002|    P00533|      O00401|          1|
|    DB00002|    P00533|      O00459|          1|
|    DB00002|    P12314|      O00526|          1|
|    DB00002|    P00533|      O00750|          1|
|    DB00002|    P00533|      O14543|          1|
|    DB00002|    P00533|      O14544|          1|
|    DB00002|    P00533|      O14818|          1|
|    DB00002|    P00533|      O14944|          1|
|    DB00002|    P00533|      O14965|          1|
|    DB00002|    P00533|      O15511|          1|
|    DB00002|    P12314|      O43491|          1|


For each drug, I count the records having the same Interactor_B. This quantity is used as a rating

In [16]:
joinedDF = joinedDF.groupBy("ID_DrugBank","Interactor_B").count()
joinedDF = joinedDF.withColumnRenamed("count", "Interactions")

joinedDF.orderBy('Interactions', ascending=False).show()

+-----------+------------+------------+
|ID_DrugBank|Interactor_B|Interactions|
+-----------+------------+------------+
|    DB12010|      P08238|          47|
|    DB12010|      P00533|          31|
|    DB12010|      Q16543|          19|
|    DB12010|      P63104|          18|
|    DB00157|      O75489|          16|
|    DB12010|      P61981|          15|
|    DB12010|      P62993|          14|
|    DB09130|      P00533|          13|
|    DB12010|      P12931|          11|
|    DB12010|      Q04917|          11|
|    DB12010|      Q12933|          11|
|    DB00157|      O43678|          10|
|    DB08515|      Q99436|          10|
|    DB15035|      P08238|          10|
|    DB12010|      P27986|          10|
|    DB08515|      O14818|          10|
|    DB12010|      P31946|          10|
|    DB11638|      P00533|          10|
|    DB08515|      P60900|          10|
|    DB12010|      P07900|           9|
+-----------+------------+------------+
only showing top 20 rows



## 4. CONVERT STRING VALUES TO INDEXES

In [17]:
drugIndexer = StringIndexer(inputCol='ID_DrugBank', outputCol='ID_DrugBank_index').fit(joinedDF)
proteinIndexer = StringIndexer(inputCol='Interactor_B', outputCol='Interactor_B_index').fit(joinedDF)

pipeline = Pipeline(stages=[drugIndexer, proteinIndexer])

indexedDF = pipeline.fit(joinedDF).transform(joinedDF)

indexedDF.show()

+-----------+------------+------------+-----------------+------------------+
|ID_DrugBank|Interactor_B|Interactions|ID_DrugBank_index|Interactor_B_index|
+-----------+------------+------------+-----------------+------------------+
|    DB07789|      P08107|           1|            630.0|              21.0|
|    DB05882|      P06401|           1|            112.0|              66.0|
|    DB00018|      Q9UKB1|           1|           2128.0|             709.0|
|    DB00431|      Q9UN36|           1|            124.0|             186.0|
|    DB01593|      Q96ST3|           1|              2.0|            1002.0|
|    DB12010|      A8MQ03|           6|              0.0|              67.0|
|    DB03319|      Q9EP80|           1|           3720.0|             605.0|
|    DB00904|      O15354|           1|           1156.0|             274.0|
|    DB01428|      P82933|           1|             35.0|             475.0|
|    DB12695|      Q9H0B6|           3|              6.0|            6370.0|

## 5. CREATING ALS MODEL

In [18]:
(training,test) = indexedDF.randomSplit([0.8, 0.2])

regParams = [0.01, 0.1]
ranks = [25]
alphas = [10.0, 20.0, 40.0, 60.0, 80.0]

aus_regParam = 0.0
aus_rank = 0
aus_alpha = 0.0
aus_rmse = 0.0

print('Creating ALS model ...')
for regParam in regParams:
    for rank in ranks:
        for alpha in alphas:
            aus_als = ALS(maxIter=10, regParam=regParam, rank=rank, alpha=alpha, userCol='ID_DrugBank_index', itemCol="Interactor_B_index", ratingCol="Interactions", coldStartStrategy="drop")
            aus_model = aus_als.fit(training)
            predictions = aus_model.transform(test)
            evaluator = RegressionEvaluator(metricName="rmse", labelCol="Interactions", predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)

            if(aus_rmse == 0.0 or rmse < aus_rmse):
                aus_regParam = regParam
                aus_rank = rank
                aus_alpha = alpha
                aus_rmse = rmse
                model = aus_model

            print("For regParam: {0}, rank: {1}, alpha: {2}, RMSE: {3}".format(regParam, rank, alpha, rmse))

print('Chosen parameters: regParam = {0}, rank = {1}, alpha = {2}'.format(aus_regParam, aus_rank, aus_alpha))

Creating ALS model ...
For regParam: 0.01, rank: 25, alpha: 10.0, RMSE: 0.3311380281996614
For regParam: 0.01, rank: 25, alpha: 20.0, RMSE: 0.3311380281996614
For regParam: 0.01, rank: 25, alpha: 40.0, RMSE: 0.3311380281996614
For regParam: 0.01, rank: 25, alpha: 60.0, RMSE: 0.3311380281996614
For regParam: 0.01, rank: 25, alpha: 80.0, RMSE: 0.3311380281996614
For regParam: 0.1, rank: 25, alpha: 10.0, RMSE: 0.3351552794004877
For regParam: 0.1, rank: 25, alpha: 20.0, RMSE: 0.33515527940048767
For regParam: 0.1, rank: 25, alpha: 40.0, RMSE: 0.3351552794004877
For regParam: 0.1, rank: 25, alpha: 60.0, RMSE: 0.3351552794004877
For regParam: 0.1, rank: 25, alpha: 80.0, RMSE: 0.33515527940048767
Chosen parameters: regParam = 0.01, rank = 25, alpha = 10.0


## 6. CREATING RECOMMENDER SYSTEM

In [19]:
print("Insert a number of recommendations per drug:")
n = int(input())

protein_recs = model.recommendForAllUsers(n)

protein_recs.show()

Insert a number of recommendations per drug:
5
+-----------------+--------------------+
|ID_DrugBank_index|     recommendations|
+-----------------+--------------------+
|             1580|[[2041, 2.622497]...|
|              471|[[2041, 8.036676]...|
|             1591|[[2041, 3.810787]...|
|             4101|[[935, 0.9972526]...|
|             1342|[[2041, 2.986436]...|
|             2122|[[2041, 4.5849857...|
|             2142|[[2041, 3.0209415...|
|              463|[[2041, 3.9053123...|
|              833|[[2041, 3.7161994...|
|             3794|[[3, 2.678913], [...|
|             1645|[[2041, 3.0027664...|
|             3175|[[2041, 1.3707684...|
|              496|[[2041, 3.7170112...|
|             2366|[[2041, 2.9704788...|
|             2866|[[2041, 2.3068767...|
|             3997|[[629, 1.2082024]...|
|              148|[[2041, 5.5285063...|
|             1088|[[2041, 4.0438128...|
|             1238|[[2041, 3.4721122...|
|             3918|[[1969, 0.998971]...|
+---------

In [20]:
flatDrugRecs = protein_recs.withColumn('proteinAndRating', explode(protein_recs.recommendations)).select('ID_DrugBank_index', 'proteinAndRating.*')

flatDrugRecs.show()

+-----------------+------------------+----------+
|ID_DrugBank_index|Interactor_B_index|    rating|
+-----------------+------------------+----------+
|             1580|              2041|  2.622497|
|             1580|              3530|  2.332521|
|             1580|                 3| 2.0062866|
|             1580|               438| 1.9677861|
|             1580|              4883| 1.8595963|
|              471|              2041|  8.036676|
|              471|              4883| 5.3744254|
|              471|              4619|  4.325515|
|              471|              3530| 3.8250642|
|              471|              5385|  3.784826|
|             1591|              2041|  3.810787|
|             1591|              4883| 2.6743968|
|             1591|              4619| 2.1584466|
|             1591|               779| 1.9743415|
|             1591|              1171| 1.9649801|
|             4101|               935| 0.9972526|
|             4101|                55| 0.9798048|


## 7. CONVERT INDEXES TO ORIGINAL STRING VALUES

In [21]:
drugString = IndexToString(inputCol='ID_DrugBank_index', outputCol='ID_DrugBank', labels=drugIndexer.labels)
proteinString = IndexToString(inputCol='Interactor_B_index', outputCol='ID_Protein', labels=proteinIndexer.labels)

convertedDrugRecs = Pipeline(stages=[drugString, proteinString]).fit(indexedDF).transform(flatDrugRecs)
convertedDrugRecs = convertedDrugRecs.select(convertedDrugRecs['ID_DrugBank'], convertedDrugRecs['ID_Protein'], convertedDrugRecs['rating'])

convertedDrugRecs.select('ID_DrugBank', 'ID_Protein', 'rating').orderBy('rating', ascending=False).show()

+-----------+----------+----------+
|ID_DrugBank|ID_Protein|    rating|
+-----------+----------+----------+
|    DB12010|    P08238| 43.217953|
|    DB12010|    P00533|  26.92041|
|    DB12010|    Q16543| 17.765347|
|    DB00157|    O75489| 15.584731|
|    DB12010|    P61981| 14.615735|
|    DB08901|    O75489| 12.472375|
|    DB01254|    O75489| 11.172857|
|    DB12010|    Q12933| 10.685572|
|    DB08515|    Q99436| 10.101418|
|    DB00157|    O43678|10.0897665|
|    DB15035|    P08238|  9.454764|
|    DB03023|    O75489| 9.3503275|
|    DB06678|    O75489|  9.260511|
|    DB08515|    O14818|  9.253752|
|    DB08515|    P25787|  9.098938|
|    DB01373|    O75489|  9.082591|
|    DB00945|    O75489|  9.059299|
|    DB09462|    O75489|  8.984226|
|    DB12695|    P08238|  8.804803|
|    DB00589|    O75489|  8.661725|
+-----------+----------+----------+
only showing top 20 rows



## 8. VALIDATING RECOMMENDER SYSTEM

In [22]:
print("Insert an id_drug: ")
id_drug = input()

print('Recommended Proteins for {0}'.format(id_drug))
convertedDrugRecs.filter(convertedDrugRecs.ID_DrugBank.isin(id_drug)).select(convertedDrugRecs['ID_Protein'], convertedDrugRecs['rating']).show(n)

print('joinedDF')
joinedDF.filter(joinedDF.ID_DrugBank.isin(id_drug)).orderBy('Interactions', 'Interactor_B', ascending=False).show()

Insert an id_drug: 
DB12010
Recommended Proteins for DB12010
+----------+---------+
|ID_Protein|   rating|
+----------+---------+
|    P08238|43.217953|
|    P00533| 26.92041|
|    Q16543|17.765347|
|    P61981|14.615735|
|    Q12933|10.685572|
+----------+---------+

joinedDF
+-----------+------------+------------+
|ID_DrugBank|Interactor_B|Interactions|
+-----------+------------+------------+
|    DB12010|      P08238|          47|
|    DB12010|      P00533|          31|
|    DB12010|      Q16543|          19|
|    DB12010|      P63104|          18|
|    DB12010|      P61981|          15|
|    DB12010|      P62993|          14|
|    DB12010|      Q12933|          11|
|    DB12010|      Q04917|          11|
|    DB12010|      P12931|          11|
|    DB12010|      P31946|          10|
|    DB12010|      P27986|          10|
|    DB12010|      P46108|           9|
|    DB12010|      P31947|           9|
|    DB12010|      P19174|           9|
|    DB12010|      P07900|           9|
| 